In [3]:
%cd ../

e:\python\atomization-energy-regression


In [4]:
from src.data_processing.molecule import get_molecule_name 
from src.utils.utils import set_seed
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error


set_seed(1706)

In [5]:
data = scipy.io.loadmat('dataset\qm7.mat')
X = data['X'] # Coulomb matrices -> 7165 x 23 x 23
T = data['T'].T.squeeze() # atomization energies Y -> 7165 x 1
Z = data['Z'] # atomic charge -> 7165 x 23
R = data['R'] # cartesian coordinates -> 7165 x 23 x 3
data_train, data_test = {}, {}
molecule_name = get_molecule_name(data)


In [4]:
v =data['X'][5577]
np.linalg.norm(v, axis=1)
idx = np.argsort(np.linalg.norm(v, axis=1))
v = v[idx,:]
np.linalg.eigvals(v)

array([ 0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        , 11.350574  +0.j        ,
       -2.685962  +0.j        ,  2.1595643 +0.j        ,
       -1.3413086 +0.j        ,  0.9704444 +0.j        ,
        0.25353876+0.j        ,  0.16964687+0.20988576j,
        0.16964687-0.20988576j, -0.2560594 +0.13355258j,
       -0.2560594 -0.13355258j, -0.2869199 +0.j        ,
       -0.06175096+0.21241368j, -0.06175096-0.21241368j,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ], dtype=complex64)

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaleZ = StandardScaler()
scaleR = StandardScaler()
def feature_engineer(data):
    X = data['X'] # Coulomb matrices -> 7165 x 23 x 23
    T = data['T'].T.squeeze() # atomization energies Y -> 7165 x 1
    Z = data['Z'] # atomic charge -> 7165 x 23
    R = data['R'] # cartesian coordinates -> 7165 x 23 x 3
    data_train, data_test = {}, {}
    molecule_name = get_molecule_name(data)
    
    Z = scaleZ.fit_transform(Z)
    R_0 = R[:,:,0]
    R_1 = R[:,:,1]
    R_2 = R[:,:,2]
    R_0 = scaleR.fit_transform(R_0)
    R_1 = scaleR.fit_transform(R_1)
    R_2 = scaleR.fit_transform(R_2)
    R = np.stack((R_0, R_1, R_2), axis=2)
    
    y = np.transpose(T)
    y_scaling_factor = np.max(np.absolute(y))
    y_scaled = y / y_scaling_factor
   
    features_vector = []
    for (x,z,r) in zip(X,Z,R):
        sorted_idx = np.argsort(np.linalg.norm(x, axis=1)) 
        sorted_coulomb_mat = x[sorted_idx, :]  # Sort rows
        sorted_coulomb_mat.sort(axis=1)
        order_x = sorted_coulomb_mat
        order_x = order_x.flatten()
        # print(nx.degree_centrality(nx.from_numpy_matrix(x)).values())
        
        feat = np.concatenate((np.linalg.eigvals(x), list(nx.degree_centrality(nx.from_numpy_matrix(x)).values()), order_x,z, r.mean(axis=0), r.std(axis=0)))
        # feat = np.concatenate((np.linalg.eigvals(x), list(nx.degree_centrality(nx.from_numpy_matrix(x)).values()), order_x,z, r.mean(axis=0), r.std(axis=0)))
        
        # feat = feat / np.linalg.norm(feat)
        features_vector.append(feat)
    return features_vector, y_scaled, y_scaling_factor

X, Y, min_max_scaler  = feature_engineer(data)
X = np.asarray(X)
Y = np.asarray(Y)

dict_values([0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
dict_values([0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.4090909090909091, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
dict_values([0.3181818181818182, 0.3181818181818182, 0.3181818181818182, 0.3181818181818182, 0.3181818181818182, 0.3181818181818182, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
dict_values([0.2272727272727273, 0.2272727272727273, 0.2272727272727273, 0.2272727272727273, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
dict_values([0.4545454545454546, 0.4545454545454546, 0.4545454545454546, 0.4545454545454546, 0.4545454545454546, 0.4545454545454546, 0.4545

In [33]:
X[0]

array([ 3.77868538e+01,  4.61345047e-01,  2.03342542e-01,  2.03281775e-01,
        2.03279957e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.72727273e-01,
        2.72727273e-01,  2.72727273e-01,  2.72727273e-01,  2.72727273e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [7]:
X.shape

(7165, 604)

In [34]:
def cross_validation(X, y, model, splits):
    mae = []
    for (idx, split) in enumerate(splits):
        mask = np.zeros(y.size, dtype=bool)
        mask[split] = True
        X_train = X[~mask]
        y_train = y[~mask]
        X_test = X[mask]
        y_test = y[mask]
        
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae.append(mean_absolute_error(y_pred, y_test))
        print(mae[-1])
    return mae
from sklearn.kernel_ridge import KernelRidge
krr_model = KernelRidge(alpha = 0.0001, kernel = "rbf", gamma=0.0001)



mae = cross_validation(X, Y, krr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.002063108529219895
0.00222399380702343
0.0021757878393263122
0.0021865152608757554
0.0021905799611916027
4.752249598324059


In [ ]:
7.828280902462109
4.757988954867747

In [71]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_regression
X_new = SelectKBest(f_regression, k=550).fit_transform(X, Y)
mae = cross_validation(X_new, Y, krr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.002358472093598374
0.0024254944526359685
0.002316148727643307
0.002479579484810037
0.002495159201691131
5.293615976230074


In [12]:
X_new.shape

(7165, 200)

In [72]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', gamma=1e-4, epsilon=1e-6)


mae = cross_validation(X, Y, svr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.0028786168777998828
0.0029050120220716416
0.0027755819021060913
0.0030067621560669567
0.002783547730923894
6.290829870043775


In [74]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
kernel = DotProduct() + WhiteKernel()
gpr_model = GaussianProcessRegressor(kernel=kernel)
mae = cross_validation(X, Y, gpr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.007340227130901871


AttributeError: 'str' object has no attribute 'decode'

In [75]:
np.mean(mae)

9.005196275938669

In [35]:
krr_model.fit(X, Y)

KernelRidge(alpha=0.0001, coef0=1, degree=3, gamma=0.0001, kernel='rbf',
            kernel_params=None)

In [36]:
krr_model.score(X, Y )

0.9999845458858015

In [29]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(learning_rate='adaptive', 
                     learning_rate_init=0.001, 
                     max_iter=100001, 
                     hidden_layer_sizes=(400, 100), 
                     activation='logistic', 
                     solver='sgd', 
                     alpha=0.0001, 
                     batch_size='auto', 
                     verbose=False, 
                     early_stopping=True, 
                     validation_fraction=0.1, 
                    #  beta_1=0.9, 
                    #  beta_2=0.999, 
                     epsilon=1e-08)
mae = cross_validation(X, Y, model, data['P'])


0.013439948102447957
0.01332065735467944
0.012199575447159534
0.013400719025581596
0.013408209029073841


In [30]:
np.mean(mae) * min_max_scaler

28.833177367600335

In [32]:
np.mean(mae) * min_max_scaler

11.935632653177851

In [35]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_depth=50,criterion='mae')
mae = cross_validation(X, Y, clf, data['P'])


0.005298723094295308
0.005418548593135123
0.005481601437321095
0.005869081531133339
0.005137643777483922


In [36]:
np.mean(mae) * min_max_scaler

11.926934353188875